In [2]:
!pip install Flask pyodbc


In [3]:
# Define your connection details
server = 'ROHITH/RO'
database = 'ro'
username = 'ROHITH/15512'



In [4]:
from flask import Flask, request, jsonify
import pyodbc
import threading

# Define your connection details
server = 'ROHITH\\RO'
database = 'ro'  # Make sure this is the correct database name

app = Flask(__name__)

# Connect to SQL Server database using Trusted Connection (Windows Authentication)
def get_db_connection():
    conn = pyodbc.connect(
        f'DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};DATABASE={database};Trusted_Connection=yes;'
    )
    return conn

@app.route('/query', methods=['POST'])
def query():
    user_query = request.json.get('query')
    response = process_query(user_query)
    return jsonify(response)

def process_query(user_query):
    conn = get_db_connection()
    cur = conn.cursor()
    # Implement logic to convert user_query to SQL and fetch results
    if "employees in the" in user_query.lower():
        department = user_query.split("department")[0].split()[-1]
        cur.execute("SELECT * FROM Employees WHERE Department = ?", (department,))
        employees = cur.fetchall()
        return [dict(zip([column[0] for column in cur.description], row)) for row in employees]
    elif "manager of the" in user_query.lower():
        department = user_query.split("department")[0].split()[-1]
        cur.execute("SELECT Manager FROM Departments WHERE Name = ?", (department,))
        manager = cur.fetchone()
        return {"Manager": manager[0]} if manager else {"Error": "Department not found"}
    elif "hired after" in user_query.lower():
        date = user_query.split("after")[-1].strip()
        cur.execute("SELECT * FROM Employees WHERE Hire_Date > ?", (date,))
        employees = cur.fetchall()
        return [dict(zip([column[0] for column in cur.description], row)) for row in employees]
    elif "total salary expense for the" in user_query.lower():
        department = user_query.split("department")[0].split()[-1]
        cur.execute("SELECT SUM(Salary) as TotalSalary FROM Employees WHERE Department = ?", (department,))
        total_salary = cur.fetchone()
        return {"TotalSalary": total_salary[0]} if total_salary else {"Error": "Department not found"}
    else:
        return {"Error": "Unsupported query"}

# Run Flask app in a separate thread.
def run_app():
    app.run(host='0.0.0.0', port=5000)

thread = threading.Thread(target=run_app)
thread.start()


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
